# 딥러닝을 활용한 삼성전자 주가예측

### 1. 필요한 모듈 불러오기 및 데이터 로딩

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'NanumGothic'

In [2]:
# df_name = pd.read_csv('시총700(코스피).txt', sep = '\t', header=None, names=['column_name'])
# df_code = pd.read_excel('기업이름.xlsx', dtype={'corp_code' : str, 'stock_code' : str})
# df_541 = df_code[df_code['corp_name'].isin(df_name['column_name'])]
# df_541.reset_index(drop=True, inplace=True)
# df_541 (예시)

In [7]:
# df_false = pd.read_excel('2017_1분기_False.xlsx')
# df_true = df_541[df_541['corp_name'].isin(df_false['corp_name']) == False]
# df_true.to_excel('2017_1분기 기업.xlsx')

In [8]:
df_samsung = pd.read_csv("005930 역사적 데이터.csv")
df_samsung.dropna(inplace=True)
df_samsung['날짜'] = df_samsung['날짜'].str.replace('[ ]', '')
df_samsung['날짜'] = df_samsung['날짜'].str.replace('년', '/')
df_samsung['날짜'] = df_samsung['날짜'].str.replace('월', '/')
df_samsung['날짜'] = df_samsung['날짜'].str.replace('일', '')
df_samsung.sort_values(by='날짜', inplace=True)
df_samsung.reset_index(drop=True, inplace=True)
df_samsung['종가'] = df_samsung['종가'].str.replace(',', '').astype(int)
df_samsung['오픈'] = df_samsung['오픈'].str.replace(',', '').astype(int)
df_samsung['고가'] = df_samsung['고가'].str.replace(',', '').astype(int)
df_samsung['저가'] = df_samsung['저가'].str.replace(',', '').astype(int)
df_samsung['거래량'] = df_samsung['거래량'].str.replace('.', '')
df_samsung['거래량'] = df_samsung['거래량'].str.replace('-', '0')
df_samsung['거래량'] = df_samsung['거래량'].str.replace('M', '0000')
df_samsung['거래량'] = df_samsung['거래량'].str.replace('K', '0').astype(int)
df_samsung['변동 %'] = df_samsung['변동 %'].str.replace('%', '').astype(float)/100
df_samsung

,날짜,종가,오픈,고가,저가,거래량,변동 %
0,2017/01/03,36480,36280,36620,36020,7340000,0.0105
1,2017/01/04,36160,36500,36520,36100,7310000,-0.0088
2,2017/01/05,35560,36060,36060,35540,10490000,-0.0166
3,2017/01/06,36200,36180,36440,36040,8380000,0.0180
4,2017/01/09,37220,36600,37500,36560,12870000,0.0282
...,...,...,...,...,...,...,...
1450,2022/03/07,70100,70000,70600,69900,1390,-0.0196
1451,2022/03/08,69500,68800,70000,68700,15720000,-0.0086
1452,2022/03/09,69500,69500,69500,69500,0,0.0000
1453,2022/03/10,71200,70900,71200,70500,2010,0.0245


In [9]:
mask = (df_samsung['종가'] <= 1000000) & (df_samsung['변동 %'] >= -0.5)
df_samsung = df_samsung[mask]
df_samsung.reset_index(drop=True, inplace=True)
df_samsung

,날짜,종가,오픈,고가,저가,거래량,변동 %
0,2017/01/03,36480,36280,36620,36020,7340000,0.0105
1,2017/01/04,36160,36500,36520,36100,7310000,-0.0088
2,2017/01/05,35560,36060,36060,35540,10490000,-0.0166
3,2017/01/06,36200,36180,36440,36040,8380000,0.0180
4,2017/01/09,37220,36600,37500,36560,12870000,0.0282
...,...,...,...,...,...,...,...
1438,2022/03/07,70100,70000,70600,69900,1390,-0.0196
1439,2022/03/08,69500,68800,70000,68700,15720000,-0.0086
1440,2022/03/09,69500,69500,69500,69500,0,0.0000
1441,2022/03/10,71200,70900,71200,70500,2010,0.0245


In [10]:
df_samsung.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1443 entries, 0 to 1442
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   날짜      1443 non-null   object 
 1   종가      1443 non-null   int64  
 2   오픈      1443 non-null   int64  
 3   고가      1443 non-null   int64  
 4   저가      1443 non-null   int64  
 5   거래량     1443 non-null   int64  
 6   변동 %    1443 non-null   float64
dtypes: float64(1), int64(5), object(1)
memory usage: 79.0+ KB


In [11]:
df_gold = pd.read_csv('금.csv')
df_gold

,날짜,종가,오픈,고가,저가,거래량,변동 %
0,2022년 03월 11일,"1,999.65","2,000.70","2,002.70","1,993.70",-,-0.05%
1,2022년 03월 10일,"2,000.65","1,994.75","2,015.00","1,975.65",-,0.63%
2,2022년 03월 09일,"1,988.20","2,060.20","2,068.50","1,981.00",-,-2.70%
3,2022년 03월 08일,"2,043.30","2,001.00","2,078.80","1,985.80",447.65K,2.37%
4,2022년 03월 07일,"1,995.90","1,978.50","2,007.50","1,964.20",372.19K,1.49%
...,...,...,...,...,...,...,...
1370,2017년 01월 06일,"1,255.70","1,256.00","1,256.00","1,256.00",0.01K,-0.51%
1371,2017년 01월 05일,"1,262.10","1,262.10","1,262.10","1,262.10",0.01K,1.20%
1372,2017년 01월 04일,"1,247.10","1,247.10","1,247.10","1,247.10",0.04K,0.19%
1373,2017년 01월 03일,"1,244.70","1,232.40","1,234.00","1,232.40",0.00K,8.14%


In [12]:
df_gold = pd.read_csv('금.csv')
df_gold = df_gold[['날짜', '종가']]
df_gold['날짜'] = df_gold['날짜'].str.replace('[ ]', '')
df_gold['날짜'] = df_gold['날짜'].str.replace('년', '/')
df_gold['날짜'] = df_gold['날짜'].str.replace('월', '/')
df_gold['날짜'] = df_gold['날짜'].str.replace('일', '')
df_gold.sort_values(by='날짜', inplace=True)
df_gold.reset_index(drop=True, inplace=True)
df_gold['종가'] = df_gold['종가'].str.replace(',', '').astype(float)
df_gold = df_gold.rename(columns={'종가' : '금'})
df_gold

,날짜,금
0,2017/01/02,1151.05
1,2017/01/03,1244.70
2,2017/01/04,1247.10
3,2017/01/05,1262.10
4,2017/01/06,1255.70
...,...,...
1370,2022/03/07,1995.90
1371,2022/03/08,2043.30
1372,2022/03/09,1988.20
1373,2022/03/10,2000.65


In [13]:
df_copper = pd.read_csv('구리.csv')
df_copper = df_copper[['날짜', '종가']]
df_copper['날짜'] = df_copper['날짜'].str.replace('[ ]', '')
df_copper['날짜'] = df_copper['날짜'].str.replace('년', '/')
df_copper['날짜'] = df_copper['날짜'].str.replace('월', '/')
df_copper['날짜'] = df_copper['날짜'].str.replace('일', '')
df_copper.sort_values(by='날짜', inplace=True)
df_copper.reset_index(drop=True, inplace=True)
df_copper = df_copper.rename(columns={'종가' : '구리'})
df_copper

,날짜,구리
0,2017/01/03,2.5150
1,2017/01/04,2.5810
2,2017/01/05,2.5670
3,2017/01/06,2.5785
4,2017/01/09,2.5725
...,...,...
1346,2022/03/07,4.7310
1347,2022/03/08,4.7100
1348,2022/03/09,4.5725
1349,2022/03/10,4.6463


In [14]:
df_gas = pd.read_csv('천연가스.csv')
df_gas = df_gas[['날짜', '종가']]
df_gas['날짜'] = df_gas['날짜'].str.replace('[ ]', '')
df_gas['날짜'] = df_gas['날짜'].str.replace('년', '/')
df_gas['날짜'] = df_gas['날짜'].str.replace('월', '/')
df_gas['날짜'] = df_gas['날짜'].str.replace('일', '')
df_gas.sort_values(by='날짜', inplace=True)
df_gas.reset_index(drop=True, inplace=True)
df_gas = df_gas.rename(columns={'종가' : '천연가스'})
df_gas

,날짜,천연가스
0,2017/01/02,3.500
1,2017/01/03,3.327
2,2017/01/04,3.267
3,2017/01/05,3.273
4,2017/01/06,3.285
...,...,...
1364,2022/03/07,4.833
1365,2022/03/08,4.527
1366,2022/03/09,4.526
1367,2022/03/10,4.664


In [15]:
df_ex = pd.read_csv('원달러환율.csv')
df_ex = df_ex[['날짜', '종가']]
df_ex['날짜'] = df_ex['날짜'].str.replace('[ ]', '')
df_ex['날짜'] = df_ex['날짜'].str.replace('년', '/')
df_ex['날짜'] = df_ex['날짜'].str.replace('월', '/')
df_ex['날짜'] = df_ex['날짜'].str.replace('일', '')
df_ex.sort_values(by='날짜', inplace=True)
df_ex.reset_index(drop=True, inplace=True)
df_ex['종가'] = df_ex['종가'].str.replace(',', '').astype(float)
df_ex = df_ex.rename(columns={'종가' : '원달러환율'})
df_ex

,날짜,원달러환율
0,2017/01/02,1203.93
1,2017/01/03,1206.82
2,2017/01/04,1196.15
3,2017/01/05,1182.19
4,2017/01/06,1202.00
...,...,...
1350,2022/03/07,1232.14
1351,2022/03/08,1233.53
1352,2022/03/09,1223.13
1353,2022/03/10,1228.74


In [16]:
df_nas = pd.read_csv('나스닥종합.csv')
df_nas = df_nas[['날짜', '종가']]
df_nas['날짜'] = df_nas['날짜'].str.replace('[ ]', '')
df_nas['날짜'] = df_nas['날짜'].str.replace('년', '/')
df_nas['날짜'] = df_nas['날짜'].str.replace('월', '/')
df_nas['날짜'] = df_nas['날짜'].str.replace('일', '')
df_nas.sort_values(by='날짜', inplace=True)
df_nas.reset_index(drop=True, inplace=True)
df_nas['종가'] = df_nas['종가'].str.replace(',', '').astype(float)
df_nas = df_nas.rename(columns={'종가' : '나스닥종합'})
df_nas

,날짜,나스닥종합
0,2017/01/03,5429.08
1,2017/01/04,5477.01
2,2017/01/05,5487.94
3,2017/01/06,5521.06
4,2017/01/09,5531.82
...,...,...
1301,2022/03/04,13313.44
1302,2022/03/07,12830.96
1303,2022/03/08,12795.55
1304,2022/03/09,13255.55


In [17]:
df_merge = pd.merge(df_samsung, df_gold, on='날짜', how='inner')
df_merge = pd.merge(df_merge, df_copper, on='날짜', how='inner')
df_merge = pd.merge(df_merge, df_gas, on='날짜', how='inner')
df_merge = pd.merge(df_merge, df_ex, on='날짜', how='inner')
df_merge = pd.merge(df_merge, df_nas, on='날짜', how='inner')
df_merge

,날짜,종가,오픈,고가,저가,거래량,변동 %,금,구리,천연가스,원달러환율,나스닥종합
0,2017/01/03,36480,36280,36620,36020,7340000,0.0105,1244.70,2.5150,3.327,1206.82,5429.08
1,2017/01/04,36160,36500,36520,36100,7310000,-0.0088,1247.10,2.5810,3.267,1196.15,5477.01
2,2017/01/05,35560,36060,36060,35540,10490000,-0.0166,1262.10,2.5670,3.273,1182.19,5487.94
3,2017/01/06,36200,36180,36440,36040,8380000,0.0180,1255.70,2.5785,3.285,1202.00,5521.06
4,2017/01/09,37220,36600,37500,36560,12870000,0.0282,1267.60,2.5725,3.103,1203.36,5531.82
...,...,...,...,...,...,...,...,...,...,...,...,...
1243,2022/03/04,71500,72700,72700,71200,13360000,-0.0192,1966.60,4.9375,5.016,1217.45,13313.44
1244,2022/03/07,70100,70000,70600,69900,1390,-0.0196,1995.90,4.7310,4.833,1232.14,12830.96
1245,2022/03/08,69500,68800,70000,68700,15720000,-0.0086,2043.30,4.7100,4.527,1233.53,12795.55
1246,2022/03/09,69500,69500,69500,69500,0,0.0000,1988.20,4.5725,4.526,1223.13,13255.55


In [18]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler_cols = ['종가', '오픈', '고가', '저가', '거래량', '금', '구리', '천연가스', '원달러환율', '나스닥종합']
scaled = scaler.fit_transform(df_merge[scaler_cols])
scaled

array([[0.01659452, 0.00404115, 0.00923483, ..., 0.38198758, 0.69810627,
        0.        ],
       [0.01082251, 0.00808229, 0.00758575, ..., 0.36956522, 0.64906233,
        0.00450963],
       [0.        , 0.        , 0.        , ..., 0.37080745, 0.58489612,
        0.00553801],
       ...,
       [0.61219336, 0.60139603, 0.55969657, ..., 0.63043478, 0.820877  ,
        0.69309564],
       [0.61219336, 0.61425422, 0.55145119, ..., 0.63022774, 0.77307409,
        0.73637607],
       [0.64285714, 0.63997061, 0.57948549, ..., 0.65879917, 0.79886008,
        0.72455957]])

In [19]:
df = pd.DataFrame(scaled, columns=scaler_cols)
df

,종가,오픈,고가,저가,거래량,금,구리,천연가스,원달러환율,나스닥종합
0,0.016595,0.004041,0.009235,0.008814,0.114348,0.000000,0.127027,0.381988,0.698106,0.000000
1,0.010823,0.008082,0.007586,0.010283,0.113881,0.002751,0.150811,0.369565,0.649062,0.004510
2,0.000000,0.000000,0.000000,0.000000,0.163421,0.019945,0.145766,0.370807,0.584896,0.005538
3,0.011544,0.002204,0.006266,0.009181,0.130550,0.012609,0.149910,0.373292,0.675951,0.008654
4,0.029942,0.009919,0.023747,0.018729,0.200499,0.026249,0.147748,0.335611,0.682203,0.009667
...,...,...,...,...,...,...,...,...,...,...
1243,0.648268,0.673035,0.604222,0.654793,0.208132,0.827487,1.000000,0.731677,0.746966,0.741823
1244,0.623016,0.623439,0.569591,0.630922,0.000022,0.861073,0.925586,0.693789,0.814488,0.696427
1245,0.612193,0.601396,0.559697,0.608887,0.244898,0.915406,0.918018,0.630435,0.820877,0.693096
1246,0.612193,0.614254,0.551451,0.623577,0.000000,0.852247,0.868468,0.630228,0.773074,0.736376


### 2. Train/Test 분할

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:].values, df['종가'].values, test_size=0.2,
                                                    random_state=2022)

In [21]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((998, 9), (250, 9), (998,), (250,))

In [22]:
input_size = X_train.shape[1]
input_size

9

In [23]:
time_step = 1

### 3. 모델 설정/정의/평가

In [24]:
import numpy as np
import tensorflow as tf

seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [25]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


In [26]:
X_train = X_train.reshape(-1, time_step, input_size)
X_test = X_test.reshape(-1, time_step, input_size)
X_train.shape, X_test.shape

((998, 1, 9), (250, 1, 9))

In [27]:
X_train.shape[0]

998

In [28]:
model = Sequential([
                    LSTM(512, input_shape = (time_step, input_size), activation = 'tanh'),
                    Dropout(0.2),
                    Dense(16, activation = 'relu'),
                    Dense(1)
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 512)               1069056   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 16)                8208      
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 1,077,281
Trainable params: 1,077,281
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.compile(optimizer = 'adam', loss = 'mse')

In [30]:
model_path = 'best-lstm.h5'
mc = ModelCheckpoint(model_path, save_best_only=True, verbose = 1)
es = EarlyStopping(patience = 10)

In [31]:
hist = model.fit(X_train, y_train, validation_split=0.2, epochs = 100, batch_size = 64,
                 callbacks = [mc, es])

Epoch 1/100
 8/13 [=================>............] - ETA: 0s - loss: 0.0938  
Epoch 1: val_loss improved from inf to 0.02177, saving model to best-lstm.h5
13/13 [==============================] - 7s 65ms/step - loss: 0.0666 - val_loss: 0.0218
Epoch 2/100
11/13 [========================>.....] - ETA: 0s - loss: 0.0191
Epoch 2: val_loss improved from 0.02177 to 0.00717, saving model to best-lstm.h5
13/13 [==============================] - 0s 14ms/step - loss: 0.0177 - val_loss: 0.0072
Epoch 3/100
11/13 [========================>.....] - ETA: 0s - loss: 0.0087
Epoch 3: val_loss improved from 0.00717 to 0.00320, saving model to best-lstm.h5
13/13 [==============================] - 0s 13ms/step - loss: 0.0080 - val_loss: 0.0032
Epoch 4/100
10/13 [======================>.......] - ETA: 0s - loss: 0.0035
Epoch 4: val_loss improved from 0.00320 to 0.00170, saving model to best-lstm.h5
13/13 [==============================] - 0s 16ms/step - loss: 0.0032 - val_loss: 0.0017
Epoch 5/100
10/13 [===

In [32]:
model.evaluate(X_test, y_test)

8/8 [==============================] - 0s 4ms/step - loss: 1.0420e-04


0.00010420027683721855